In [1]:
from IPython.display import clear_output, display
from IPython.core.debugger import set_trace
from knockknock import telegram_sender

TELEGRAM_API_KEY  = '966193511:AAH-8JXV6XZHhHgBIN-M7xD6SIFeSBi8-kk'
CHAT_ID = 232512362
root = 'data/'
models  = 'models/'
submission = 'submission/'

KEY = ['building_id', 'meter', 'site_id']
KEY_NAME = 'building_id__meter__key'


print("Helper ipython script loaded")

Helper ipython script loaded


In [2]:
%%html

<style>
    body {
          font-family: Helvetica, Times New Roman, sans-serif;
    }
    
    h1,h2, h3,h4,h5,h6 {
        font-family: Rockwell, Times New Roman, sans-serif;
    }
</style>

In [3]:
import pandas as pd
import numpy as np

import sys
from tqdm.auto import tqdm
tqdm.pandas(desc="")
import warnings
#warnings.filterwarnings("ignore")

from time import time
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

pd.options.display.float_format = '{:,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
init_notebook_mode(connected=True)
import os
import timeit


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

print("Basic Data Analysis tools was loaded")

Basic Data Analysis tools was loaded


In [4]:
from bayes_opt import BayesianOptimization
from hyperopt import hp
from hyperopt import fmin, tpe, space_eval
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from hyperopt.pyll import scope

import math
from sklearn.preprocessing import LabelEncoder
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters ,ComprehensiveFCParameters
from sklearn.model_selection import KFold, cross_validate, cross_val_predict
from tsfresh.utilities.dataframe_functions import make_forecasting_frame, impute

from sklearn.model_selection import *
import lightgbm as lgb
#import xgboost as xgb

def batch_prediction(model, samples, n = 20):
    batches = np.array_split(samples, n)
    agg = []
    
    for batch in batches:
        agg.append(*(model.predict(batches).values.tolist()))
        
    return agg

def create_submission(model, test_data, submission_name, send_submission=False, message="Enter model name"):
    result = batch_prediction(model, test_data)
    
    submission_df = pd.DataFrame() 
    submission_df['row_id'] = np.arange(len(preds))
    submission_df['meter_reading'] = preds
    submission_df.to_csv(submission + submission_name, index=False)
   
    if send_submission:
        !kaggle competitions submit -c ashrae-energy-prediction -f {submission_name} -m {message}
        
time_cols = ['month_datetime', 'weekofyear_datetime',
             'dayofyear_datetime', 'hour_datetime', 
             'day_week', 'day_month_datetime',
             'week_month_datetime']

feat_cols = ['building_id', 'meter', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'age', 'month_datetime', 'weekofyear_datetime',
       'dayofyear_datetime', 'hour_datetime', 'day_week', 'day_month_datetime',
       'week_month_datetime', KEY_NAME]

categoricals =  ['building_id', 'meter','site_id',
       'primary_use', 'year_built', 'age', 'month_datetime', 'weekofyear_datetime',
       'dayofyear_datetime', 'hour_datetime', 'day_week', 'day_month_datetime',
       'week_month_datetime', KEY_NAME]        

In [5]:
%macro -q __ipy 1 2
%macro -q __da 3
%macro -q __ml 4

In [6]:
%store __ipy
%store __da
%store __ml

Stored '__ipy' (Macro)
Stored '__da' (Macro)
Stored '__ml' (Macro)
